Vecteurs techniques (tools_list + topics_list)

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
# --- Charger la DB enrichie ---
df = pd.read_json("github_data_enriched.jsonl", lines=True)

# --- Construire vocabulaire technique ---
all_tech = set()
for tools in df['tools_list']:
    all_tech.update(tools)
for topics in df['topics_list']:
    all_tech.update(topics)

vocab_list = sorted(list(all_tech))  # ordre fixe

repo_texts = df.apply(
    lambda row: " ".join(row["tools_list"] + row["topics_list"]),
    axis=1
)

vectorizer = TfidfVectorizer()
X_tech = vectorizer.fit_transform(repo_texts)

# ✅ SAUVEGARDE
np.save("X_tech.npy", X_tech.toarray())

print("TF-IDF sauvegardé :", X_tech.shape)

TF-IDF sauvegardé : (1349, 4745)


Vecteurs sémantiques (full_text embeddings)

In [6]:
import pandas as pd
import torch
import numpy as np
from sentence_transformers import SentenceTransformer

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device :", device)

model = SentenceTransformer("all-MiniLM-L6-v2", device=device)

df = pd.read_json("github_data_enriched.jsonl", lines=True)
texts = df["full_text"].fillna("").tolist()

embeddings = model.encode(
    texts,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True
)

# ✅ SAUVEGARDE
np.save("embeddings_semantic.npy", embeddings)

print("Embeddings sauvegardés :", embeddings.shape)

Device : cuda


Batches:   0%|          | 0/43 [00:00<?, ?it/s]

Embeddings sauvegardés : (1349, 384)


Vecteurs numériques (features supplémentaires)

In [7]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

numeric_features = df[
    ["log_stars", "fork_ratio", "recency_days"]
].fillna(0)

scaler = MinMaxScaler()
X_numeric = scaler.fit_transform(numeric_features)

# ✅ SAUVEGARDE
np.save("X_numeric.npy", X_numeric)

print("Features numériques sauvegardées :", X_numeric.shape)


Features numériques sauvegardées : (1349, 3)


Vecteur hybride final

In [8]:
import numpy as np
from sklearn.preprocessing import normalize
from scipy.sparse import csr_matrix

# 🔄 Charger depuis disque
embeddings = np.load("embeddings_semantic.npy")
X_tech = csr_matrix(np.load("X_tech.npy"))
X_numeric = np.load("X_numeric.npy")

# Normalisation
embeddings = normalize(embeddings)
X_tech = normalize(X_tech)
X_numeric = normalize(X_numeric)

# Pondérations
W_SEM = 0.6
W_TECH = 0.3
W_NUM = 0.1

# Fusion
X_hybrid = np.hstack([
    embeddings * W_SEM,
    X_tech.toarray() * W_TECH,
    X_numeric * W_NUM
])

# ✅ SAUVEGARDE FINALE
np.save("X_hybrid.npy", X_hybrid)

print("🔥 Vecteur hybride prêt")
print("Shape :", X_hybrid.shape)


🔥 Vecteur hybride prêt
Shape : (1349, 5132)


Stockage

In [9]:
import os

# Dossier principal pour stocker les fichiers
DATA_DIR = "data/processed"

os.makedirs(DATA_DIR, exist_ok=True)

print(f"✅ Dossier de stockage prêt : {DATA_DIR}")

✅ Dossier de stockage prêt : data/processed


In [10]:
import pandas as pd

# Charger la DB enrichie si pas déjà en mémoire
df = pd.read_json("github_data_enriched.jsonl", lines=True)

# Sélection des colonnes utiles pour stockage
metadata_cols = [
    "databaseId", "nameWithOwner", "url", "target_domain",
    "stargazerCount", "forkCount", "recency_days",
    "tools_list", "topics_list"
]

metadata_df = df[metadata_cols]

# Sauvegarde JSONL
metadata_file = os.path.join(DATA_DIR, "metadata.jsonl")
metadata_df.to_json(metadata_file, orient='records', lines=True, force_ascii=False)

print(f"✅ Métadonnées sauvegardées : {metadata_file}")

✅ Métadonnées sauvegardées : data/processed\metadata.jsonl


In [11]:
import numpy as np

# embeddings : numpy array (N, 384)
# exemple : embeddings déjà calculés avec SentenceTransformer
# embeddings = model.encode(...)

embeddings_file = os.path.join(DATA_DIR, "embeddings_semantic.npy")
np.save(embeddings_file, embeddings)

print(f"✅ Embeddings sémantiques sauvegardés : {embeddings_file}")


✅ Embeddings sémantiques sauvegardés : data/processed\embeddings_semantic.npy


In [12]:
from scipy.sparse import save_npz

# X_tech : matrice sparse
# vectorizer.vocabulary_ : dictionnaire vocabulaire

# Sauvegarder matrice sparse
X_tech_file = os.path.join(DATA_DIR, "X_tech.npz")
save_npz(X_tech_file, X_tech)

# Sauvegarder vocabulaire JSON
import json
vocab_file = os.path.join(DATA_DIR, "vocab_tech.json")
with open(vocab_file, "w", encoding="utf-8") as f:
    json.dump(vectorizer.vocabulary_, f, ensure_ascii=False, indent=2)

print(f"✅ Vecteurs TF-IDF sauvegardés : {X_tech_file}")
print(f"✅ Vocabulaire sauvegardé : {vocab_file}")


✅ Vecteurs TF-IDF sauvegardés : data/processed\X_tech.npz
✅ Vocabulaire sauvegardé : data/processed\vocab_tech.json


In [13]:
# X_numeric : numpy array
numeric_file = os.path.join(DATA_DIR, "X_numeric.npy")
np.save(numeric_file, X_numeric)

print(f"✅ Features numériques sauvegardées : {numeric_file}")


✅ Features numériques sauvegardées : data/processed\X_numeric.npy


In [14]:
from sklearn.preprocessing import normalize
from scipy.sparse import csr_matrix

# Normalisation
embeddings_norm = normalize(embeddings)
X_tech_norm = normalize(X_tech)
X_numeric_norm = normalize(X_numeric)

# Pondération (modifiable)
W_SEM = 0.6
W_TECH = 0.3
W_NUM = 0.1

X_hybrid = np.hstack([
    embeddings_norm * W_SEM,
    X_tech_norm.toarray() * W_TECH,
    X_numeric_norm * W_NUM
])

hybrid_file = os.path.join(DATA_DIR, "X_hybrid.npy")
np.save(hybrid_file, X_hybrid)

print(f"✅ Vecteur hybride sauvegardé : {hybrid_file}")
print("Shape :", X_hybrid.shape)


✅ Vecteur hybride sauvegardé : data/processed\X_hybrid.npy
Shape : (1349, 5132)


FAISS

In [15]:
import numpy as np

DATA_DIR = "data/processed"
hybrid_file = f"{DATA_DIR}/X_hybrid.npy"

X_hybrid = np.load(hybrid_file)
print("✅ Vecteur hybride chargé")
print("Shape :", X_hybrid.shape)


✅ Vecteur hybride chargé
Shape : (1349, 5132)


In [18]:
import faiss

d = X_hybrid.shape[1]  # dimension des vecteurs
index_cpu = faiss.IndexFlatL2(d)  # distance L2

# Ajouter tous les vecteurs
index_cpu.add(X_hybrid.astype(np.float32))
print("✅ Index FAISS CPU créé")
print("Nombre de vecteurs dans l'index :", index_cpu.ntotal)

# Sauvegarder l'index
faiss.write_index(index_cpu, "index/faiss_index_cpu.bin")
print("✅ Index FAISS CPU sauvegardé : index/faiss_index_cpu.bin")


✅ Index FAISS CPU créé
Nombre de vecteurs dans l'index : 1349
✅ Index FAISS CPU sauvegardé : index/faiss_index_cpu.bin


In [21]:
import faiss
import numpy as np  # je suppose que X_hybrid est un np.array

# Dimension des vecteurs
d = X_hybrid.shape[1]

# Créer l'index FAISS CPU (compatible Windows)
index_flat = faiss.IndexFlatL2(d)

# Ajouter les vecteurs
index_flat.add(X_hybrid.astype(np.float32))
print("✅ Index FAISS CPU créé")
print("Nombre de vecteurs dans l'index :", index_flat.ntotal)

# Sauvegarder l'index CPU
faiss.write_index(index_flat, "index/faiss_index_cpu.bin")
print("✅ Index FAISS sauvegardé : index/faiss_index_cpu.bin")


✅ Index FAISS CPU créé
Nombre de vecteurs dans l'index : 1349
✅ Index FAISS sauvegardé : index/faiss_index_cpu.bin
